In [33]:
import pandas as pd
import random
import string
from datetime import datetime

In [34]:
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_ctr.csv', header = 0)

In [35]:
resultados_top_ctr.head()

,log,advertiser_id,product_id
0,2023-04-23,03KNVBO915KY2ZPGA57J,ffkh8f
1,2023-04-23,03KNVBO915KY2ZPGA57J,0zmyxz
2,2023-04-23,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-23,03KNVBO915KY2ZPGA57J,2b9w5m
4,2023-04-23,03KNVBO915KY2ZPGA57J,3calwd


In [36]:
datetime.today().strftime("%Y-%m-%d")

'2023-05-05'

In [37]:
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_ctr.csv', header = 0)
resultados_top_product = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_product.csv', header = 0)

In [38]:
union = resultados_top_ctr.merge(resultados_top_product, 
                                 on = ['advertiser_id','log'], 
                                 how= 'outer')

In [39]:
resultados_top_ctr.set_index(['log','advertiser_id']).head()

product_id
log        advertiser_id                  
2023-04-23 03KNVBO915KY2ZPGA57J     ffkh8f
           03KNVBO915KY2ZPGA57J     0zmyxz
           03KNVBO915KY2ZPGA57J     28ycmv
           03KNVBO915KY2ZPGA57J     2b9w5m
           03KNVBO915KY2ZPGA57J     3calwd

In [40]:
#resultados_top_ctr.set_index(['log','advertiser_id'])[(resultados_top_ctr.log == '2023-04-23') & (resultados_top_ctr.advertiser_id == '03KNVBO915KY2ZPGA57J')]

In [41]:
resultados_top_ctr.groupby('advertiser_id').head(7).head(10)

,log,advertiser_id,product_id
0,2023-04-23,03KNVBO915KY2ZPGA57J,ffkh8f
1,2023-04-23,03KNVBO915KY2ZPGA57J,0zmyxz
2,2023-04-23,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-23,03KNVBO915KY2ZPGA57J,2b9w5m
4,2023-04-23,03KNVBO915KY2ZPGA57J,3calwd
5,2023-04-23,03KNVBO915KY2ZPGA57J,4bu13h
6,2023-04-23,03KNVBO915KY2ZPGA57J,4hzw8j
20,2023-04-23,1OW4LWJSBJK6E1D9FDH7,a8hf9j
21,2023-04-23,1OW4LWJSBJK6E1D9FDH7,023w1n
22,2023-04-23,1OW4LWJSBJK6E1D9FDH7,0xfk8v


In [42]:
resultados_top_ctr.log.min()

'2023-04-23'

# Creamos una base completa con historia

In [93]:
import datetime
from datetime import timedelta
import numpy as np

In [94]:
# Levantamos la data
usuarios = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/advertiser_ids')
total_logs = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/ads_views')
total_products = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/product_views.csv')

# Usuarios inactivos
usuarios_inactivos = usuarios.tail(5)['advertiser_id'].to_list()

In [103]:
print(total_logs.date.min())
print(total_logs.date.max())

# Días únicos
dias_unicos = list(total_logs.date.unique())
print("Cantidad de días únicos: ", len(dias_unicos))

2023-04-01
2023-05-30
Cantidad de días únicos:  60


In [173]:
%%time

resultados_totales_top_ctr = pd.DataFrame(columns = ['log', 'advertiser_id', 'product_id'])
resultados_totales_top_product = pd.DataFrame(columns = ['log', 'advertiser_id', 'product_id'])

for i in range(1,60):

    # Vamos a simular la base que resulte de aplicar todos los días
    yesterday = dias_unicos[i-1]

    # Nos quedamos con la base filtrada
    datos_top_ctr = total_logs[(~total_logs['advertiser_id'].isin(usuarios_inactivos)) & (total_logs['date'] == yesterday)]
    datos_top_pro = total_products[(~total_products['advertiser_id'].isin(usuarios_inactivos)) & (total_products['date'] == yesterday)]

    ########################
    # Calculamos el TOPCTR #
    ########################

    # Hacemos las agrupaciones
    top_ctr = datos_top_ctr.groupby(['advertiser_id','product_id'])['type'].value_counts().reset_index().pivot(index = ['advertiser_id','product_id'], columns='type', values='count').reset_index()
    top_ctr.fillna(0, inplace = True)

    # Hacemos el ratio
    top_ctr['ctr'] = round(top_ctr['click']  / top_ctr['impression'],2)

    # Casteamos los valores infinitos como un ctr del 100%, asumiento un typo en el registro de la impresión
    top_ctr['ctr'] = np.where(np.isinf(top_ctr['ctr']),1,top_ctr['ctr'])

    # Ordenamos por los valores más altos
    top_ctr = top_ctr.sort_values(by = ['advertiser_id', 'ctr'], ascending = [True, False])

    # Obtenemos los 20 productos con mayor conversión por advertiser
    resultado_top_ctr = top_ctr.groupby('advertiser_id').head(20)

    # Nos quedamos con las columnas de interés
    resultado_top_ctr['log'] = dias_unicos[i]
    resultado_top_ctr = resultado_top_ctr[['log','advertiser_id','product_id']]

    # Appendeamos
    resultados_totales_top_ctr = pd.concat([resultados_totales_top_ctr, resultado_top_ctr], ignore_index=True)

    ############################
    # Calculamos el TopProduct #
    ############################    

    # Hacemos las agrupaciones
    datos_top_pro = datos_top_pro.groupby(['advertiser_id', 'product_id']).count().reset_index()
    
    # Ordenamos
    top_product = datos_top_pro.sort_values(by = ['advertiser_id', 'date'], ascending = [True, False])    

    # Nos quedamos con los 20 productos más vistos por advertiser
    resultados_top_product = top_product.groupby('advertiser_id').head(20)

    # Nos quedamos con las columnas de interés
    resultados_top_product['log'] = dias_unicos[i]
    resultados_top_product = resultados_top_product[['log','advertiser_id','product_id']]

    # Appendeamos
    resultados_totales_top_product = pd.concat([resultados_totales_top_product, resultados_top_product], ignore_index=True)

<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

CPU times: user 1.49 s, sys: 0 ns, total: 1.49 s
Wall time: 1.48 s


<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [175]:
display(resultados_totales_top_product.shape)
display(resultados_totales_top_ctr.shape)

(23600, 3)

(23600, 3)

In [168]:
# Guardamos los resultados
resultados_totales_top_ctr.to_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_ctr.csv', index = False, header = True)

resultados_totales_top_product.to_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_ctr.csv', index = False)


In [ ]:
fechas_ordenadas = sorted(list(resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J']['log'].unique()))

In [209]:
fechas_ordenadas = sorted(list(resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J']['log'].unique()), reverse = True)[0:7]
print(fechas_ordenadas)
#resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J'].sort_values(by = ['log'], ascending = [False]).head()

['2023-05-30', '2023-05-29', '2023-05-28', '2023-05-27', '2023-05-26', '2023-05-25', '2023-05-24']


# Ya tenemos la base completa con toda la historia

In [184]:
resultados_totales_top_ctr.tail()

,log,advertiser_id,product_id
23595,2023-05-30,SOVPFK3BBWKTQM9HOHWJ,j7ua0b
23596,2023-05-30,SOVPFK3BBWKTQM9HOHWJ,keol8b
23597,2023-05-30,SOVPFK3BBWKTQM9HOHWJ,ltpkiv
23598,2023-05-30,SOVPFK3BBWKTQM9HOHWJ,m76e67
23599,2023-05-30,SOVPFK3BBWKTQM9HOHWJ,mg3tnw


In [212]:
# Levantamos los resultados de cada modelo
# TopCTR
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_ctr.csv', header = 0)
resultados_top_ctr = resultados_top_ctr[(resultados_top_ctr['advertiser_id'] == 'SOVPFK3BBWKTQM9HOHWJ')]

In [ ]:

	
# TopProduct
resultados_top_product = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_product.csv', header = 0)
resultados_top_product = resultados_top_product[(resultados_top_product['advertiser_id'] == ADV)]

# Compactamos por el advertiser que se requiere
resultados_adv = pd.concat([resultados_top_ctr, resultados_top_product], ignore_index=True)

# Agarramos los últimos 7 días
fechas_ordenadas = sorted(list(resultados_adv['log'].unique()), reverse = True)[0:7]

# Filtramos los productos recomendados para el advertiser en esos días
filtrado = resultados_adv[resultados_adv['log'].isin([fechas_ordenadas]),]['product_id'].unique()